In [1]:
import sys
import numpy as np
import pandas as pd
import lightgbm as lg
from tqdm import tqdm
import pandas as pd
from pathlib import Path

# from importlib import reload


import catboost
from catboost.utils import get_gpu_device_count

sys.path.append("../../scorer/")
import orderbook_fast as ob

# from  solution import get_simple_features_from_orderbook, get_simple_deals_features
# from  solution import get_simple_deals_features

from my_orderbook import MyOrderBook
catboost_myob = MyOrderBook()

SIDE_BID = 0 
SIDE_ASK = 1

print(f"curdir: {Path.cwd()}")

curdir: /home/sergey/mnt/st1500/Usr/Sergey/TheJob/Challenges/wunder_summer/wunder_challenge/examples/catboost_local


## Собираем датасет для тренировки модели

In [ ]:
def collect_dataset(data_path):
    '''
        Собираем датасет
    '''

    global catboost_myob
    event_player = ob.EventPlayer(data_path)
    orderbook = ob.OrderBook()

    X = []
    Y = []

    for ev in tqdm(event_player.iget_events(),
                    total=len(event_player),
                    desc="collecting dataset"):

        if ev.action == ob.Action.DEAL:
            catboost_myob.set_last_deal(ev)
        elif ev.action == ob.Action.NEW_CHUNK:
            catboost_myob.clear()

        orderbook.apply_event(ev)
        if ev.need_prediction:
            features = catboost_myob.get_features(ev, orderbook)

            X.append(features)
            Y.append(ev.Y)

    print(f"Dataset collected: len(X) = {len(X)}")
    return pd.DataFrame(X), pd.DataFrame(Y)

# X_train, Y_train = collect_dataset("../../data/very_small_A.npz")
# X_test, Y_test = collect_dataset("../../data/very_small_B.npz")

X_train_large, Y_train_large = collect_dataset("../../data/train.npz")
X_train, Y_train = collect_dataset("../../data/train_small_A.npz")
X_test, Y_test = collect_dataset("../../data/train_small_B.npz")

X_train_large.to_pickle("X_train_large.pickle")
Y_train_large.to_pickle("Y_train_large.pickle")

X_train.to_pickle("X_train_small_A.pickle")
Y_train.to_pickle("Y_train_small_A.pickle")

X_test.to_pickle("X_train_small_B.pickle")
Y_test.to_pickle("Y_train_small_B.pickle")

print("X_train.shape  == ", X_train.shape)

collecting dataset:  81%|████████  | 25551365/31735888 [2:39:26<55:39, 1851.86it/s]  

## Обучаем модель градиентного бустинга

In [ ]:
%%time
def train_catboost(X_train, Y_train, X_test, Y_test):

    task_type = "CPU" #  if (get_gpu_device_count() == 0) else "GPU"
    langevin = False if task_type == "GPU" else True
    print(f"catboost training with {task_type}...")
    print("X_train.shape = ", X_train.shape)
    print("get_gpu_device_count() = ", get_gpu_device_count())

    train_pool = catboost.Pool(X_train, Y_train, cat_features=catboost_myob.cat_features)     #    , cat_features=cat_features)

    grid = {'learning_rate': [0.03],
            # 'score_function': ["Cosine", "L2", "NewtonL2"],
            'depth': [4, 5, 6],
            'l2_leaf_reg': [2, 3, 5, 8, 11],
            # 'nan_mode': ["Min", "Max"],
            # 'fold_len_multiplier': np.linspace(1.1, 3, 3),
            # 'bagging_temperature': np.linspace(1, 3000, 3),

    }

    model = catboost.CatBoostClassifier(

        bootstrap_type="Bayesian",
        # scale_pos_weight=en.scale_pos_weight,
        # scale_pos_weight=1,
        loss_function = "Logloss",
        eval_metric="AUC:hints=skip_train~false",
        langevin=langevin,
        custom_metric=["Recall", "Precision", "Accuracy", "F1", "Kappa", "MCC"],
        # use_best_model=True,
        iterations=5,
        od_type="Iter",
        od_wait=30,
        # rsm=0.5,
        # random_seed=100,
        task_type=task_type,
        boosting_type='Ordered',
        logging_level="Verbose",
        train_dir="grid",

    )

    grid_search_result = model.grid_search(
        grid,
        X=train_pool,
        stratified=True,
        cv=3,
        search_by_train_test_split=False,
        plot=False      # PLOT,

    )


    params = model.get_params()
    for param in params.items():
        print(param)

    return grid_search_result, model

grid_search_result, clf = train_catboost(X_train, Y_train, X_test, Y_test)
# clf = train_classifier(X_train, Y_train, X_test, Y_test)

## Тестируем получившийся классификатор

In [ ]:
def process_event_and_predict_proba(ev, orderbook):

    if ev.action == ob.Action.DEAL:
        catboost_myob.set_last_deal(ev)
    elif ev.action == ob.Action.NEW_CHUNK:
        catboost_myob.clear()



    if not ev.need_prediction:
        return None
    
    features = catboost_myob.get_features(ev, orderbook)
    proba = clf.predict_proba([features])[0, 1]
    return proba

from scorer import Scorer

# scoring = Scorer("../../data/train_small_C.npz")
scoring = Scorer("../../data/very_small_B.npz")
roc_auc, (true_ys, pred_probas) = scoring.score(process_event_and_predict_proba)

## Сохраним модель, и решение для отправки готово

In [ ]:
# Сохраним нашу модель
# clf.booster_.save_model("wunder.model")
clf.save_model("wunder.model", format="cbm")
print(f"curdir: {Path.cwd()}")
'''Посмотрите код файла solution.py. 
Он использует те же функции что и этот ноутбук, но уже готов к отправке на серверю. 
Попробуйте создать архив с файлами solution.py и wunder.model и отправить их на проверку.'''